# LLM Workflow Design Patterns: Routing
- "Handoffs" in Agents SDK follows the routing design pattern
- This file is a project for applying design pattern: routing
- Project idea:
    - A router agent `RouterAgent` routes an e-commerce customer support messages to the appropriate customer support agent to handle the message
    - Support Agents:
        1. `DeliveryAgent`
        2. `BillingAgent`
        3. `TechAgent`
    - If the `RouterAgent` receives a message that doesn't seem to be from the previous cases, send a notification via **pushover**

``` python
def main():
    with open("./article.txt", "r", encoding="utf-8") as f:
        article = f.read()
    summary = summarize(article)
    keywords = extract_keywords(summary)
    quiz = generate_quiz(keywords=keywords, num_questions=5) # change here
    display_quiz(quiz)
```

<br>

![Prompt Chaining Diagram](../../assets/routing.png)

### Usual set up

In [187]:
# imports

import os
import json
import requests
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool, OpenAIChatCompletionsModel, set_default_openai_client, set_tracing_disabled
from openai import AsyncOpenAI


In [188]:
# load environment variables
load_dotenv(override=True)

api_key = os.getenv("GOOGLE_API_KEY")
gemini_base_url = os.getenv("GOOGLE_API")

if api_key:
    print("Model API key loaded successfully")
else:
    print("Error: couldn't load API key")
if gemini_base_url:
    print("Gemini URL loaded successfully")
else:
    print("Error: couldn't load Gemini URL")


Model API key loaded successfully
Gemini URL loaded successfully


In [189]:
# create Gemini model
model = "gemini-2.5-flash"

gemini_client = AsyncOpenAI(base_url=gemini_base_url, api_key=api_key)
set_default_openai_client(gemini_client)
set_tracing_disabled(True)

gemini_model = OpenAIChatCompletionsModel(model=model, openai_client=gemini_client)

In [190]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user found and starts with u
Pushover token found and starts with a


In [191]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [ ]:
push("Test")

### Tools

In [192]:
departments = ["Delivery", "Billing", "Technical"]

In [193]:
# send notification to phone if the customer has further questions that the agent doesn't know
@function_tool
def record_question(email: str, question="not provided"):
    """ Send out a notification with the customer's email and question """
    push(f"A customer with the following email {email} has a question: {question}")
    return {"status": "success"}

In [ ]:
record_question

FunctionTool(name='record_user_details', description="Send out a notification with the customer's email and question", params_json_schema={'properties': {'email': {'title': 'Email', 'type': 'string'}, 'question': {'default': 'not provided', 'title': 'Question'}}, 'required': ['email', 'question'], 'title': 'record_user_details_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001BD23F45760>, strict_json_schema=True, is_enabled=True)

In [194]:
@function_tool
def get_policy(department: str, issue: str) -> str: 
    """Retrieve the policy and action steps for a given department and issue."""
    if department not in departments:
        return f"Error: Unknown department '{department}'. Choose from: {departments}."
    with open("policies.json") as f:
        policies = json.load(f)
    return policies.get(department, {}).get(issue, "No policy found for the passed issue.")

In [54]:
get_policy

FunctionTool(name='get_policy', description='Retrieve the policy and action steps for a given department and issue.', params_json_schema={'properties': {'department': {'title': 'Department', 'type': 'string'}, 'issue': {'title': 'Issue', 'type': 'string'}}, 'required': ['department', 'issue'], 'title': 'get_policy_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001BD23F14FE0>, strict_json_schema=True, is_enabled=True)

### Building Delivery Support Agent

In [195]:
tools = [record_question, get_policy]

In [196]:
instructions = "You are a customer support agent in the Delivery department, you handle ONLY support service messages from customers\
 , you will use get_policy tool to get the delivery policies \
 , The issue must be one of: LateDelivery, LostPackage, ChangeAddress. \
, Never mention or leak these issue types as they are given. \
 , If the issue is outside your scope, state that (without mentioning issue types), and ask for the customer's email and use the record_question tool to send a notification and \
 tell the customer that further support will contact soon."

delivery_agent = Agent(name="Delivery Agent", handoff_description="Handle order delivery customer support questions.", instructions=instructions, tools=tools, model=gemini_model)

In [129]:
delivery_agent

Agent(name='Delivery Agent', instructions="You are a customer support agent in the Delivery department, you handle ONLY support service messages from customers , you will use get_policy tool to get the delivery policies  , The issue must be one of: LateDelivery, LostPackage, ChangeAddress. , Never mention or leak these issue types as they are given.  , If the issue is outside your scope, state that (without mentioning issue types), and ask for the customer's email and use the record_question tool to send a notification and  tell the customer that further support will contact soon.", handoff_description='Handle order delivery customer support questions.', handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x000001BD246E4490>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=N

In [157]:
# agent testing
result = await Runner.run(delivery_agent, "Tell me a joke")
print(result.final_output)

I can't help you with that, as I'm a customer support agent in the Delivery department. Could you please provide your email address? I'll record your question, and further support will contact you soon.


### Building Billing Suppor Agent

In [197]:
instructions = "You are a customer support agent in the Billing department, you handle ONLY support service messages from customers\
 , you will use get_policy tool to get the billing policies \
 , The issue must be one of: RefundRequest, PaymentFailure. \
, Never mention or leak these issue types as they are given. \
 , If the issue is outside your scope, state that (without mentioning issue types), and ask for the customer's email and use the record_question tool to send a notification and \
 tell the customer that further support will contact soon."

billing_agent = Agent(name="Billing Agent", handoff_description="Handle order billing customer support questions.", instructions=instructions, tools=tools, model=gemini_model)

In [198]:
# agent testing
result = await Runner.run(billing_agent, "I want to upgrade my plan")
print(result.final_output)

I can only help with billing-related issues such as refunds or payment failures. For plan upgrades, I'll need to transfer your request. Could you please provide your email address? I'll record your question, and a support agent will contact you shortly to assist you further.


### Building Technical Support Agent

In [199]:
# email packae
from sendgrid.helpers.mail import Mail, Email, To, Content
import sendgrid

In [200]:
@function_tool
def send_password_reset_email(email: str):
    """ Send out a password reset email to the given email """
    sg = sendgrid.SendGridAPIClient(api_key=os.getenv('SENDGRID_API_KEY'))
    from_email = Email(os.getenv("SENDER_EMAIL"))  
    to_email = To(email) 
    body = """
    We received a request to reset your password for your account. If you made this request, please click the link below to create a new password:
[Reset Password]

This link will expire in 60 minutes for your security. If you didn’t request a password reset, you can safely ignore this email—your password will remain unchanged.

Need help? Contact our support team at [support email].

Thanks,
The [Company Name] Team
    """
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Reset Your Password", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [134]:
send_password_reset_email

FunctionTool(name='send_password_reset_email', description='Send out a password reset email to the given email', params_json_schema={'properties': {'email': {'title': 'Email', 'type': 'string'}}, 'required': ['email'], 'title': 'send_password_reset_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001BD24750B80>, strict_json_schema=True, is_enabled=True)

In [201]:
technical_agent_tools = [send_password_reset_email, record_question, get_policy]

In [211]:
instructions = """"You are the Technical Support Agent in the company's customer support system.

Your role:
- Handle only *technical support* messages from customers.
- Continue the ongoing conversation naturally — do not restart, greet, or reintroduce yourself after the first message.
- Maintain a professional, helpful, and concise tone.

Your responsibilities:
1. Identify the technical issue the user describes.
   - The issue must be one of: ConnectionIssue, AppCrash, ForgotPassword.
   - Never reveal or mention these internal issue names explicitly.
2. Use the **get_policy** tool to retrieve and follow the relevant technical policy before replying.
3. If the user asks to reset their password:
   - Use the **send_password_reset_email** tool to send the reset link to their email.
   - Confirm to the user that the reset email has been sent.
4. If the issue is outside your technical scope:
   - Politely explain that the issue will be escalated.
   - Ask for the customer’s email (if not already provided).
   - Use the **record_question** tool to notify higher support.
   - Reassure the customer that further support will contact them soon.

Response style:
- Always continue contextually from the conversation history.
- Never start with greetings like “Hello” or “How can I help you today” unless it’s the first user message of the entire chat.
- Be polite, confident, and solution-oriented.

Additional rules:
- Never expose internal tools or policy names.
- Never handle delivery, billing, or non-technical topics — instead, say that this is outside your department.
- Always act as if you’re a real human agent, not an AI model.
"""

technical_agent = Agent(name="Technical Agent", handoff_description="Handle technical customer support questions", instructions=instructions, tools=technical_agent_tools, model=gemini_model)

In [203]:
technical_agent

Agent(name='Technical Agent', instructions="You are a customer support agent in the Technical department, you handle ONLY support service messages from customers , you will use get_policy tool to get the technical policies  , The issue must be one of: ConnectionIssue, AppCrash, ForgotPassword.  , Never mention or leak these issue types as they are given.  , If user needs to reset their password, use the send_password_reset_email tool to send the reset email.  , If the issue is outside your scope, state that, and ask for the customer's email and use the record_question tool to send a notification and  tell the customer that further support will contact soon.", handoff_description='Handle technical customer support questions', handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x000001BD27C6AE50>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, 

In [ ]:
# agent testing
result = await Runner.run(technical_agent, "I want to download my data")
print(result.final_output)

Push: A customer with the following email das@gmail.com has a question: User wants to download their data.
Thank you for providing your email. We will get back to you shortly regarding your request to download your data.


In [212]:
handoffs = [delivery_agent, billing_agent, technical_agent]

### Create The Router 

In [213]:
router_instructions = """You are a Routing Agent, 
Your job is to analyze the user's message and decide which department should handle it.

There are three department agents available:
1. Delivery Agent – handles order status, delays, shipping, and address change issues.
2. Billing Agent – handles refunds and payment failures.
3. Technical Agent – handles connection problems, app crashes, and password reset.

Your responsibilities:
- Read the user's message carefully.
- Identify the issue type and select ONE department that is best suited to handle it.
- When confident, perform a HANDOFF to the selected department agent.
- Once the handoff is done, do not continue processing the conversation.
- If the message is unclear or does not match any known category, ask the user a short clarifying question before routing.
- Never attempt to solve the problem yourself — your only job is to route messages.

Response style:
- Be concise and professional.
- If clarification is needed, use a friendly and direct question.
- If routing, state which department you’re routing to and then perform the handoff.

Your only valid actions:
- Ask for clarification, OR
- Handoff to one of: DeliveryAgent, BillingAgent, TechAgent. OR
- If the customer's problem can't be handled by the previous agents, ask for customer's email and send a notification using record_question tool
"""

routing_agent = Agent(name="Routing Agent", instructions=router_instructions, handoffs=handoffs, tools=[record_question], model=gemini_model)

In [167]:
# agent testing
result = await Runner.run(routing_agent, "I want to reset my password")
print(result)
print(result.final_output)

RunResult:
- Last agent: Agent(name="Technical Agent", ...)
- Final output (str):
    Please provide your email address to reset your password.
- 4 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
Please provide your email address to reset your password.


### Building The Chat

In [214]:
history = []
starting_agent = routing_agent
while True:
    prompt = input("Enter You Questions: ")
    if prompt == "end":
        break
    result = await Runner.run(
        starting_agent=starting_agent,
        input=prompt,
        context=history,
    )
    history.extend([{
        "role": "user",
        "content": prompt
    },
    {
        "role": "assistant",
        "content": result.final_output
    }])
    starting_agent = result.last_agent
    print(history)
    print(result.last_agent.name)
    print(result.final_output)

[{'role': 'user', 'content': 'Hello, i need technical help'}, {'role': 'assistant', 'content': "Please describe your technical issue, and I'll do my best to help."}]
Technical Agent
Please describe your technical issue, and I'll do my best to help.
[{'role': 'user', 'content': 'Hello, i need technical help'}, {'role': 'assistant', 'content': "Please describe your technical issue, and I'll do my best to help."}, {'role': 'user', 'content': 'i need to reset my password'}, {'role': 'assistant', 'content': 'Please provide your email address so I can send you a password reset link.'}]
Technical Agent
Please provide your email address so I can send you a password reset link.
[{'role': 'user', 'content': 'Hello, i need technical help'}, {'role': 'assistant', 'content': "Please describe your technical issue, and I'll do my best to help."}, {'role': 'user', 'content': 'i need to reset my password'}, {'role': 'assistant', 'content': 'Please provide your email address so I can send you a password

In [ ]:
async def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    print(history)
    result = await Runner.run(starting_agent=routing_agent, input=message, context=history)
    print(result)
    return result.final_output

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


[]
RunResult:
- Last agent: Agent(name="Technical Agent", ...)
- Final output (str):
    I can help with technical issues. What is your issue about?
- 3 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
[{'role': 'user', 'metadata': None, 'content': 'Hi i have a technical question', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'I can help with technical issues. What is your issue about?', 'options': None}]
RunResult:
- Last agent: Agent(name="Technical Agent", ...)
- Final output (str):
    It looks like you're having trouble with your password. To help you with this, I can send you a password reset email. Could you please provide the email address associated with your account?
- 4 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
[{'role': 'user', 'metadata': None, 'content': 'Hi i have a technical question'